In [20]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [21]:
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.dtypes

Downcasting floats.


player_id                       object
player_name                     object
player_display_name             object
position                        object
position_group                  object
headshot_url                    object
recent_team                     object
season                           int32
week                             int32
season_type                     object
opponent_team                   object
completions                      int32
attempts                         int32
passing_yards                  float32
passing_tds                      int32
interceptions                  float32
sacks                          float32
sack_yards                     float32
sack_fumbles                     int32
sack_fumbles_lost                int32
passing_air_yards              float32
passing_yards_after_catch      float32
passing_first_downs            float32
passing_epa                    float32
passing_2pt_conversions          int32
pacr                     

In [22]:
# Create a DF for WR's and TE's
wr_df_two = weekly_nfl[(weekly_nfl["position"] == "WR") | (weekly_nfl["position"] == "TE")]
new_columns = ['opponent_team','recent_team','week','player_display_name','fantasy_points', 'fantasy_points_ppr','receiving_yards','receiving_yards_after_catch','receptions','targets','receiving_tds','receiving_air_yards','receiving_first_downs','receiving_epa','target_share','air_yards_share','racr','wopr','headshot_url']
wr_df = wr_df_two[new_columns]
wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df.head()

,opponent_team,recent_team,week,player_display_name,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr,headshot_url
4,GB,NO,3,Jimmy Graham,6.8,7.8,8.0,0.0,1,1,1,8.0,1.0,2.708027,0.029412,0.029091,1.000000,0.064481,https://static.www.nfl.com/image/private/f_aut...
9,NE,NYJ,3,Randall Cobb,1.2,2.2,12.0,10.0,1,2,0,6.0,1.0,0.441822,0.071429,0.037500,2.000000,0.133393,https://static.www.nfl.com/image/private/f_aut...
15,KC,DET,1,Marvin Jones,-1.2,0.8,8.0,4.0,2,6,0,50.0,0.0,-6.195231,0.187500,0.230415,0.160000,0.442540,https://static.www.nfl.com/image/private/f_aut...
23,ATL,CAR,1,Adam Thielen,1.2,3.2,12.0,3.0,2,2,0,9.0,1.0,0.757165,0.064516,0.039474,1.333333,0.124406,https://static.www.nfl.com/image/private/f_aut...
24,NO,CAR,2,Adam Thielen,13.4,20.4,54.0,24.0,7,9,1,59.0,4.0,2.174741,0.290323,0.279621,0.915254,0.631218,https://static.www.nfl.com/image/private/f_aut...


In [23]:
#create df for sum D vs WR
wr_sum_df = wr_df.groupby('opponent_team').sum().reset_index()
wr_sum_df.set_index('opponent_team', inplace=True)

In [24]:
team_col = ['player_display_name', 'recent_team']
team_df = weekly_nfl[team_col]
team_df.set_index('player_display_name', inplace=True)

In [25]:
# create elbow curve
# Overall Individual Seseason
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(wr_sum_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="NFL WR Average Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [26]:
# OVR MODEL
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = data.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [27]:
# Create ML for traditional stats
fantasy_points = wr_sum_df.groupby("opponent_team").sum()['fantasy_points']
fantasy_points_ppr = wr_sum_df.groupby("opponent_team").sum()['fantasy_points_ppr']
yards = wr_sum_df.groupby("opponent_team").sum()['receiving_yards']
receptions = wr_sum_df.groupby("opponent_team").sum()['receptions']
targets = wr_sum_df.groupby("opponent_team").sum()['targets']
tds = wr_sum_df.groupby("opponent_team").sum()['receiving_tds']
target_share = wr_sum_df.groupby("opponent_team").sum()['target_share']

trad_ml_df = pd.DataFrame({
    "fantasy_points": fantasy_points,
    "fantasy_points_ppr": fantasy_points_ppr,
    "yards": yards,
    "receptions": receptions,
    "targets": targets,
    "tds":tds
    
})

trad_ml_df.head()

,fantasy_points,fantasy_points_ppr,yards,receptions,targets,tds
opponent_team,,,,,,
ARI,76.500000,131.500000,690.0,55,83,1
ATL,80.400002,128.399994,495.0,48,74,5
BAL,71.599998,138.600006,597.0,67,104,2
BUF,52.799999,86.800003,374.0,34,46,2
CAR,63.200001,107.200005,552.0,44,65,1


In [28]:
#create df for sum D vs WR
wr_sum_df_adv = wr_df_adv.groupby('opponent_team').sum().reset_index()
wr_sum_df_adv.set_index('opponent_team', inplace=True)

In [29]:
# create ml 5 ks
d_trad_df = get_clusters(5, trad_ml_df)
d_trad_df.head()

,fantasy_points,fantasy_points_ppr,yards,receptions,targets,tds,class
opponent_team,,,,,,,
ARI,76.500000,131.500000,690.0,55,83,1,4
ATL,80.400002,128.399994,495.0,48,74,5,0
BAL,71.599998,138.600006,597.0,67,104,2,1
BUF,52.799999,86.800003,374.0,34,46,2,3
CAR,63.200001,107.200005,552.0,44,65,1,0


In [30]:
#make xlsx with ml group

d_trad_df.to_excel('wr_d_ml_trad_sum.xlsx', index=True)

In [31]:
# Create ML Model to Group into 5 clusters Defense's vs WR/TE

# DF with Advanced Stats

# Create a DF for WR's and TE's
wr_df_adv = weekly_nfl[(weekly_nfl["position"] == "WR") | (weekly_nfl["position"] == "TE")]
new_columns = ['opponent_team','recent_team','week','player_display_name','fantasy_points', 'fantasy_points_ppr','receiving_yards','receiving_yards_after_catch','receptions','targets','receiving_tds','receiving_air_yards','receiving_first_downs','receiving_epa','target_share','air_yards_share','racr','wopr','headshot_url']
wr_df_adv = wr_df_adv[new_columns]
wr_df_adv_two= wr_df_adv[wr_df_adv['fantasy_points_ppr'] != 0]
wr_df_adv.head()

,opponent_team,recent_team,week,player_display_name,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr,headshot_url
4,GB,NO,3,Jimmy Graham,6.8,7.8,8.0,0.0,1,1,1,8.0,1.0,2.708027,0.029412,0.029091,1.00,0.064481,https://static.www.nfl.com/image/private/f_aut...
7,BUF,NYJ,1,Randall Cobb,0.0,0.0,0.0,0.0,0,1,0,10.0,0.0,-3.924131,0.055556,0.133333,0.00,0.176667,https://static.www.nfl.com/image/private/f_aut...
8,DAL,NYJ,2,Randall Cobb,0.0,0.0,0.0,0.0,0,1,0,15.0,0.0,-1.523066,0.040000,0.056604,0.00,0.099623,https://static.www.nfl.com/image/private/f_aut...
9,NE,NYJ,3,Randall Cobb,1.2,2.2,12.0,10.0,1,2,0,6.0,1.0,0.441822,0.071429,0.037500,2.00,0.133393,https://static.www.nfl.com/image/private/f_aut...
15,KC,DET,1,Marvin Jones,-1.2,0.8,8.0,4.0,2,6,0,50.0,0.0,-6.195231,0.187500,0.230415,0.16,0.442540,https://static.www.nfl.com/image/private/f_aut...


In [32]:
# Create ML for advanced stats
fantasy_points = wr_sum_df_adv.groupby("opponent_team").sum()['fantasy_points']
fantasy_points_ppr = wr_sum_df_adv.groupby("opponent_team").sum()['fantasy_points_ppr']
yards = wr_sum_df_adv.groupby("opponent_team").sum()['receiving_yards']
wr_epa = wr_sum_df_adv.groupby("opponent_team").sum()['receiving_epa']
racr = wr_sum_df_adv.groupby("opponent_team").sum()['racr']
air_yards_share = wr_sum_df_adv.groupby("opponent_team").sum()['air_yards_share']
target_share = wr_sum_df_adv.groupby("opponent_team").sum()['target_share']

adv_ml_df = pd.DataFrame({
    "fantasy_points": fantasy_points,
    "fantasy_points_ppr": fantasy_points_ppr,
    "yards": yards,
    "wr_epa": wr_epa,
    "target_share": target_share,
    "racr":racr,
    "target_share": target_share
    
})

adv_ml_df.head()

,fantasy_points,fantasy_points_ppr,yards,wr_epa,target_share,racr
opponent_team,,,,,,
ARI,76.500000,131.500000,690.0,14.103231,2.507363,18.016464
ATL,80.400002,128.399994,495.0,6.334526,2.597419,8.236010
BAL,71.599998,138.600006,597.0,11.696054,2.515419,22.288631
BUF,52.799999,86.800003,374.0,0.569273,2.047393,11.136776
CAR,63.200001,107.200005,552.0,22.658941,2.155369,17.522192


In [33]:
# 5 Clusters

# create ml 5 ks
d_adv_df = get_clusters(5, adv_ml_df)
d_adv_df.head()

,fantasy_points,fantasy_points_ppr,yards,wr_epa,target_share,racr,class
opponent_team,,,,,,,
ARI,76.500000,131.500000,690.0,14.103231,2.507363,18.016464,1
ATL,80.400002,128.399994,495.0,6.334526,2.597419,8.236010,0
BAL,71.599998,138.600006,597.0,11.696054,2.515419,22.288631,3
BUF,52.799999,86.800003,374.0,0.569273,2.047393,11.136776,4
CAR,63.200001,107.200005,552.0,22.658941,2.155369,17.522192,3


In [34]:
#make xlsx with ml group for Advanced

d_adv_df.to_excel('wr_d_ml_adv_sum.xlsx', index=True)